<a href="https://colab.research.google.com/github/Suryaday/Simple-chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import os
import io
import numpy as np
import random
import string
import warnings
warnings.filterwarnings('ignore')
os.chdir('/')
import bs4 as bs
import urllib.request
import re

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Breast_cancer')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)
#print(article_text)

In [ ]:
article_sentences = nltk.sent_tokenize(article_text)
article_words = nltk.word_tokenize(article_text)

In [ ]:
#Stemming the Words
wnlemmatizer = nltk.stem.WordNetLemmatizer()


def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [ ]:
#For Vectorizing and finding Vector similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def generate_response(user_input):
    tennisrobo_response = ''
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(article_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        tennisrobo_response = tennisrobo_response + "I am sorry, I could not understand you"
        return tennisrobo_response
    else:
        tennisrobo_response = tennisrobo_response + article_sentences[similar_sentence_number] + "\n" + article_sentences[similar_sentence_number+1] \
                              + "\n" + article_sentences[similar_sentence_number+2] + "\n" + article_sentences[similar_sentence_number+3]
        return tennisrobo_response

In [ ]:
pip install googletrans

     |████████████████████████████████| 61kB 2.3MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 972kB 8.8MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=e8462cd16a3256a7ac4e57435319c5d17143a7bcd86948acdc355915cd1a4464
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=0b7e1874135a6a5e012efe17b0b0847e898910b34914e5b4bc92f73b5e90e3c3
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
continue_dialogue = True
print("Hello, I am your friend Aasha. You can ask me any question regarding Breast Cancer:")
while(continue_dialogue == True):
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            print("Aasha: Most welcome")
        else:
            if generate_greeting_response(human_text) != None:
                print("Aasha: " + generate_greeting_response(human_text))
            else:
                print("Aasha: ", end="")
                lst = generate_response(human_text).split('\n')
                for i in lst:
                  print(i.title())
                  trans = translator.translate(i, dest='hi')
                  print(trans.text)

                article_sentences.remove(human_text)
    else:
        continue_dialogue = False
        print("Aasha: Good bye and take care of yourself...")

Hello, I am your friend Aasha. You can ask me any question regarding Breast Cancer:
What is breast cancer?
Aasha:  Breast Cancer Is Cancer That Develops From Breast Tissue.
स्तन कैंसर, कैंसर है जो स्तन के ऊतकों से विकसित होता है।
Signs Of Breast Cancer May Include A Lump In The Breast, A Change In Breast Shape, Dimpling Of The Skin, Fluid Coming From The Nipple, A Newly-Inverted Nipple, Or A Red Or Scaly Patch Of Skin.
स्तन कैंसर के संकेतों में स्तन में एक गांठ, स्तन के आकार में बदलाव, त्वचा का धुंधलापन, निप्पल से निकलने वाला तरल पदार्थ, एक नव-उल्टे निप्पल या त्वचा का लाल या टेढ़ा पैच शामिल हो सकता है।
In Those With Distant Spread Of The Disease, There May Be Bone Pain, Swollen Lymph Nodes, Shortness Of Breath, Or Yellow Skin.
उन लोगों में जो रोग के दूर फैलते हैं, हड्डियों में दर्द, सूजन लिम्फ नोड्स, सांस की तकलीफ या पीली त्वचा हो सकती है।
Risk Factors For Developing Breast Cancer Include Being Female, Obesity, A Lack Of Physical Exercise, Alcoholism, Hormone Replacement Therapy During

KeyboardInterrupt: ignored